In [3]:
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
headers = {
    'authority': 'finviz.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'nl-BE,nl-NL;q=0.9,nl;q=0.8,en-US;q=0.7,en;q=0.6'}

In [18]:
'''loop to get full table per ticker'''
news_tables = {}
financial = {}
tickers = ['TDOC','RDS-A', 'EURN']

for ticker in tickers:
    print(f"sending request for {ticker}")
    params = (
    ('t', ticker),
    )
    response = requests.get('https://finviz.com/quote.ashx', headers=headers, params=params) 
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response.content, "html.parser")
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find('table', class_='fullview-news-outer').find_all('tr')
    snapshot_table = html.find('table', class_='snapshot-table2')
    # Add the table to our dictionary
    news_tables[ticker] = news_table
    financial[ticker] = snapshot_table

sending request for TDOC
sending request for RDS-A
sending request for EURN


In [196]:
'''Get final dict with news'''
new_dict = {}

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Create new list per iteration
    parsed_news = []
    intermediate_dict = {}
    # Iterate through all tr tags in 'news_table'
    for x in news_table:
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        #ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([date, text])
    #intermediate dictionary
    intermediate_dict['news'] = parsed_news
    # Append full list to dict
    new_dict[file_name] = intermediate_dict


In [197]:
'''Get financial dict'''
financial_dict = {}
for ticker, values in financial.items():
    table_dark_row = values.find_all('tr', class_="table-dark-row")
    values_dict = {}
    intermediate_dict = {}
    #table_dark_row = financial['TDOC'].find_all('tr', class_="table-dark-row")
    for i in table_dark_row:
        keys = i.find_all('td', class_="snapshot-td2-cp")
        values = i.find_all('td', class_="snapshot-td2")
        for keys, values in zip(keys, values):
            values_dict[keys.text] =  values.text
    intermediate_dict['fundamentals'] = values_dict
    new_dict[ticker]['fundamentals'] = values_dict


In [170]:
d = {}

for key in set(list(new_dict.keys()) + list(financial_dict.keys())):
    try:
        d.setdefault(key,[]).append(dict(new_dict[key]))     
    except KeyError:
        pass

    try:
        d.setdefault(key,[]).append(financial_dict[key])          
    except KeyError:
        pass
